In [23]:
import numpy as np # type: ignore
import tensorflow as tf # type: ignore
from keras import datasets, layers, models # type: ignore
import matplotlib.pyplot as plt # type: ignore
import pandas as pd
import os
import cv2 as cv
from keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input

In [2]:
# Establish filepaths 
all_0 = os.getcwd() + "/C-NMC_Leukemia/training_data/fold_0/all"
all_1 = os.getcwd() + "/C-NMC_Leukemia/training_data/fold_1/all"
all_2 = os.getcwd() + "/C-NMC_Leukemia/training_data/fold_2/all"

hem_0 = os.getcwd() + "/C-NMC_Leukemia/training_data/fold_0/hem"
hem_1 = os.getcwd() + "/C-NMC_Leukemia/training_data/fold_1/hem"
hem_2 = os.getcwd() + "/C-NMC_Leukemia/training_data/fold_2/hem"
print(all_0)

/Users/isaacbilsel/Documents/GW_Projects/ML/CSCI-4364-Final-Project/C-NMC_Leukemia/training_data/fold_0/all


In [3]:
def get_path_image(folder):
    image_paths = []
    image_fnames = os.listdir(folder) 
    for img_id in range(len(image_fnames)):
        img = os.path.join(folder,image_fnames[img_id])
        image_paths.append(img)
    
    return image_paths

In [32]:
train_data = []
test_data = []

# for i in [all_0,all_1,all_2,hem_0,hem_1,hem_2]:
for i in [all_0,hem_0]:   # Start training on just the first folder
    paths = get_path_image(i)
    train_data.extend(paths)
print(len(train_data))

# Test data is the last folder for now 
# (because idk how to parse the csv file to get the test labels)
for i in [all_2,hem_2]:   
    paths = get_path_image(i)
    test_data.extend(paths)
print(len(test_data))

3527
3553


In [ ]:
# Create pandas dataframe and add labels
# Training Data
tr_data = {"img_data":train_data,
        "labels":[np.nan for x in range(len(train_data))]}
df_train = pd.DataFrame(tr_data)
df_train["labels"][0:2379] = 1 # ALL:  2397 in folder 0, 7272 in all 3 folders
df_train["labels"][2379:] = 0 # HEM  1130 in folder 1, 3389 in all 3 folders
df_train["labels"] = df_train["labels"].astype("int64")

# Test Data
ts_data = {"img_data":test_data,
        "labels":[np.nan for x in range(len(test_data))]}
df_test = pd.DataFrame(ts_data)
df_test["labels"][0:2457] = 1 # ALL:  2457 in folder 0, 7272 in all 3 folders
df_test["labels"][2457:] = 0 # HEM  1096 in folder 1, 3389 in all 3 folders
df_test["labels"] = df_test["labels"].astype("int64")

train_x = np.asarray(df_train["img_data"])
train_y = np.asarray(df_train["labels"])
test_x = np.asarray(df_test["img_data"])
test_y = np.asarray(df_test["labels"])

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10661 entries, 0 to 10660
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   img_data  10661 non-null  object 
 1   labels    3509 non-null   float64
dtypes: float64(1), object(1)
memory usage: 166.7+ KB


In [34]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   img_data  3553 non-null   object
 1   labels    3553 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 55.6+ KB


In [ ]:
image = cv.imread(df_train["img_data"][100])
plt.imshow(image)

In [46]:
'''
Creat Model Structure
For high accuracy, it seems like we need to build on a pretrained CNN
The two top solutions on kaggle build on efficientnet and Resnet
We could build on VGG, which is another strong option
VGG19 extracts important features from the dataset
'''

pretrained_model = VGG19(weights='imagenet', input_shape = (224, 224, 3), include_top=False, pooling="avg")

model = Sequential([
    pretrained_model,
    # MaxPooling2D((2,2) , strides = 2),
    Flatten(),
    # BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, activation='softmax'),
    Dropout(rate= 0.45, seed= 123),
    Dense(2, activation='softmax')])
model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 512)               20024384  
                                                                 
 flatten_10 (Flatten)        (None, 512)               0         
                                                                 
 dense_20 (Dense)            (None, 256)               131328    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 2)                 514       
                                                                 
Total params: 20,156,226
Trainable params: 20,156,226
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Train the model
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose=1,factor=0.6, min_lr=0.000001)

history = model.fit(train_x,train_y, batch_size = 64 , epochs = 25 , validation_data = (test_x, test_y), callbacks = [learning_rate_reduction])

Epoch 1/25


ValueError: in user code:

    File "/Users/isaacbilsel/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/isaacbilsel/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/isaacbilsel/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/isaacbilsel/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/isaacbilsel/opt/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/isaacbilsel/opt/anaconda3/lib/python3.9/site-packages/keras/engine/